In [1]:
!pip install firebase_admin

  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 8.7 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.62.3 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.29.1 which is incompatible.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.29.1 which is incompatible.


# JSON with all the Data

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import json

# Initialize Firebase Admin SDK
firebase_cred_path = "perkcup-689f9-firebase-adminsdk-yuf7v-c17798a18e.json"  # Replace with your Firebase Admin JSON
if not firebase_admin._apps:
    cred = credentials.Certificate(firebase_cred_path)
    firebase_admin.initialize_app(cred)

# Firestore client
db = firestore.client()

def fetch_all_documents(collection_ref):
    """Recursively fetch all documents and their subcollections."""
    data = {}
    try:
        for doc in collection_ref.stream():
            doc_id = doc.id
            print('doc :' , doc_id)
            doc_data = doc.to_dict()
            data[doc_id] = doc_data

            # Check for subcollections
            subcollections = db.collection(collection_ref.id).document(doc_id).collections()
            for subcollection in subcollections:
                subcollection_name = subcollection.id
                data[doc_id][subcollection_name] = fetch_all_documents(subcollection)

        return data
    except Exception as e:
        print(f"Error fetching documents: {e}")
        return {}

def fetch_full_database():
    """Fetch the full Firestore database content."""
    database_content = {}
    try:
        # Fetch all top-level collections
        collections = db.collections()
        for collection in collections:
            collection_name = collection.id
            print(collection_name)
            database_content[collection_name] = fetch_all_documents(collection)
        return database_content
    except Exception as e:
        print(f"Error fetching database: {e}")
        return {}

import json
from datetime import datetime

def save_to_file(data, filename):
    """Save the fetched data to a JSON file."""
    def default_serializer(obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # Convert datetime to ISO 8601 string
        if hasattr(obj, '__dict__'):
            return obj.__dict__  # Serialize objects with `__dict__` attribute
        return str(obj)  # Fallback to string representation

    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4, default=default_serializer)
        print(f"Database content saved to {filename}")
    except Exception as e:
    
        print(f"Error saving to file: {e}")

if __name__ == "__main__":
    print("Fetching full Firestore database content...")
    full_database_content = fetch_full_database()
    print("Saving...")
    # Save the fetched data to a JSON file
    save_to_file(full_database_content, "firestore_full_database.json")

Fetching full Firestore database content...
algorithmResults
doc : 04H5VopZ31XSdobrSHbL
doc : 0JliZttapu9TjzDcZNDj
doc : 4eM2sIR8PIrmM0MA18QZ
doc : 7SDCxRxvmyXsHnlhQvj2
doc : 9kKmFciHxgv9MQ3MPQ1H
doc : Aql5tL2Qd5zwdwWGxIrz
doc : BrcwkbmSic6yqcQojKap
doc : DY3VzNxLNSmya3rNolzZ
doc : FOPUwMNhSMlf5vYkgPrg
doc : FXv6eyiMTlblYz1wMs9w
doc : IlSn8jSCY3SJzRReo8lY
doc : ItMKv3krhYfbT0ecSDtp
doc : KouS7YLfdqZ012ztOrXU
doc : Qa2nW96rqyiUE54EFMS1
doc : SW8cGeQo3yCMVCULNm2C
doc : W9oLaiubEc6VcKjP5VKp
doc : YZc4WTBB4Z1fExHdyVH2
doc : aZ3UzBEiwTiBkLmx5yMu
doc : fTWyfx0o5ooLYlTIpZLn
doc : iCk99TZx5aImZfBGy68l
doc : jDijtWjZGCPpxVvKQDMV
doc : k84xFNV0gHQnQ2NPZx8k
doc : zzPwMG2yZ1FEFOxWVyrA
blocks
doc : block_1
doc : block_2
doc : block_3
doc : block_4
doc : block_5
doc : block_6
doc : block_7
doc : block_8
olayers
doc : Aaron, Rai
doc : Abraham, Ancer
doc : Ackerman, Derek
doc : Adam, Hadwin
doc : Adam, Schenk
doc : Adam, Scott
doc : Adam, Svensson
doc : Adamonis, Brad
doc : Adrian, Meronk
doc : Adrien

In [ ]:
import pandas as pd
import json

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)
file_path = "firestore_full_database.json"
x = load_json(file_path)
x["scores"]["21"]

In [17]:
import pandas as pd
import json

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

def extract_tournaments_data(data):
    """
    Extract tournaments data from the loaded JSON, including details about players and other event information.
    """
    tournaments_data = []
    for year, tours in data.get('tournaments', {}).items():
        for tour, events in tours.items():
            for event_id, event_details in events.items():
                # General tournament details
                tournament_info = {
                    'year': year,
                    'tour': tour,
                    'event_name': event_details.get('event_name'),
                    'event_id': event_id,
                    'isUpcoming': event_details.get('isUpcoming'),
                    'finish_date': event_details.get('finish_date'),
                    'last_fetched': event_details.get('last_fetched')
                }
                
                # Include players' data if available
                players = event_details.get('website_info', {}).get('players', [])
                for player in players:
                    player_info = {
                        'player_name': player.get('Name'),
                        'finish_position': player.get('Finish_Position'),
                        'perk_cup_value': player.get('PerkCup_Value'),
                        'sg_total': player.get('SG_Total'),
                        'sg_app': player.get('SG_APP'),
                        'sg_ott': player.get('SG_OTT'),
                        'sg_putt': player.get('SG_PUTT'),
                        'sg_arg': player.get('SG_ARG'),
                        'field': player.get('Field'),
                        'comment': player.get('Comment')
                    }
                    # Combine tournament info with player info
                    tournaments_data.append({**tournament_info, **player_info})
                    
                # If no players, just append the tournament info
                if not players:
                    tournaments_data.append(tournament_info)

    return pd.DataFrame(tournaments_data)

def extract_players_data(data):
    """
    Extract players data from the loaded JSON.
    """
    players_data = []
    for year, tours in data.get('tournaments', {}).items():
        for tour, events in tours.items():
            for event_id, event_details in events.items():
                for player in event_details.get('Results', {}).get('scores', []):
                    player_data = {
                        'event_id': event_details.get('event_id'),
                        'player_name': player.get('player_name'),
                        'dg_id': player.get('dg_id'),
                        'final_position': player.get('fin_text')
                    }

                    # Add round details
                    for round_num in range(1, 5):
                        round_key = f'round_{round_num}'
                        round_details = player.get(round_key, {})
                        player_data.update({
                            f'round_{round_num}_score': round_details.get('score'),
                            f'round_{round_num}_sg_app': round_details.get('sg_app'),
                            f'round_{round_num}_course_par': round_details.get('course_par'),
                            f'round_{round_num}_driving_acc': round_details.get('driving_acc'),
                            f'round_{round_num}_teetime': round_details.get('teetime'),
                            f'round_{round_num}_sg_total': round_details.get('sg_total'),
                            f'round_{round_num}_sg_arg': round_details.get('sg_arg'),
                            f'round_{round_num}_gir': round_details.get('gir'),
                            f'round_{round_num}_sg_putt': round_details.get('sg_putt'),
                            f'round_{round_num}_prox_fw': round_details.get('prox_fw'),
                            f'round_{round_num}_scrambling': round_details.get('scrambling'),
                            f'round_{round_num}_course_num': round_details.get('course_num'),
                            f'round_{round_num}_great_shots': round_details.get('great_shots'),
                            f'round_{round_num}_course_name': round_details.get('course_name'),
                            f'round_{round_num}_sg_ott': round_details.get('sg_ott'),
                            f'round_{round_num}_start_hole': round_details.get('start_hole'),
                            f'round_{round_num}_prox_rgh': round_details.get('prox_rgh'),
                            f'round_{round_num}_poor_shots': round_details.get('poor_shots'),
                            f'round_{round_num}_driving_dist': round_details.get('driving_dist'),
                            f'round_{round_num}_sg_t2g': round_details.get('sg_t2g')
                        })
                    players_data.append(player_data)
    return pd.DataFrame(players_data)

def extract_players_info(data):
    """
    Extract players information from the loaded JSON.
    """
    players_data = data.get("players", {})
    return pd.DataFrame.from_dict(players_data, orient='index')

def save_to_csv(df, file_name):
    """
    Save a DataFrame to a CSV file.
    """
    df.to_csv(file_name, index=False)
    print(f"CSV file '{file_name}' has been created.")

def main():
    # Load the JSON data
    data = load_json('firestore_full_database.json')

    # Extract tournaments and players data
    tournaments_df = extract_tournaments_data(data)
    #players_df = extract_players_data(data)
    players_info_df = extract_players_info(data)

    # Save to CSV
    save_to_csv(tournaments_df, 'CSV DATA/tournaments.csv')
    #save_to_csv(players_df, 'events.csv')
    save_to_csv(players_info_df, 'CSV DATA/players_info.csv')


main()

import json
import pandas as pd

# Function to load JSON data
def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

# Load JSON file
file_path = "firestore_full_database.json"  # Replace with your file path
data = load_json(file_path)

# Extract scores data
scores = data.get("scores", {})

# Process all events
rows = []
for event_id, event_data in scores.items():
    years = event_data.get("years", {})
    for year, year_details in years.items():
        event_name = year_details.get("event_name", "N/A")
        tour = year_details.get("tour", "N/A")
        tournament_id = year_details.get("tournament_id", "N/A")
        date = year_details.get("date", "N/A")
        actual_results = year_details.get("actual_results", [])

        for player in actual_results:
            player_name = player.get("name", "N/A")
            total_score = player.get("total_score", 0)
            fin_text = player.get("fin_text", "N/A")
            strokes_gained = player.get("strokes_gained", 0)
            percup_value = player.get("perkcup_value", 0)  # Include `perkcup_value` with default 0
            rounds = player.get("rounds", {})
            round_1 = rounds.get("round_1", {})
            round_2 = rounds.get("round_2", {})
            round_3 = rounds.get("round_3", {})
            round_4 = rounds.get("round_4", {})

            # Add player details to rows
            rows.append({
                "Event ID": event_id,
                "Year": year,
                "Event Name": event_name,
                "Tour": tour,
                "Tournament ID": tournament_id,
                "Date": date,
                "Player Name": player_name,
                "Total Score": total_score,
                "Final Position": fin_text,
                "Strokes Gained": strokes_gained,
                "Perkcup Value": percup_value,
                "Round 1 Score": round_1.get("score", 0),
                "Round 1 SG Total": round_1.get("sg_total", 0),
                "Round 2 Score": round_2.get("score", 0),
                "Round 2 SG Total": round_2.get("sg_total", 0),
                "Round 3 Score": round_3.get("score", 0),
                "Round 3 SG Total": round_3.get("sg_total", 0),
                "Round 4 Score": round_4.get("score", 0),
                "Round 4 SG Total": round_4.get("sg_total", 0),
            })

# Create a DataFrame and save it as a CSV
output_file = "CSV DATA/all_event_scores.csv"
df = pd.DataFrame(rows)
df.to_csv(output_file, index=False)
print(f"All event scores data saved to {output_file}")
import json
import csv

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

def save_blocks_with_names(blocks, output_file):
    """
    Extract data for each name in blocks and save it to a CSV with specified columns.
    """
    rows = []
    for block_id, block_details in blocks.items():
        # Extract block-level fields
        block_number = block_details.get("blockNumber", "N/A")
        tournament_ids = ", ".join([str(tid) if tid is not None else "N/A" for tid in block_details.get("tournamentIds", [])])
        block_draft_positions = block_details.get("blockDraftPositions", "N/A")

        # Extract player-specific fields
        previous_tournaments = block_details.get("previousBlockTournamentsPlayed", {})
        previous_points = block_details.get("previousBlockTotalPerkCupPoints", {})
        next_field_points = block_details.get("nextBlockHistoricalFieldPoints", {})
        previous_positions = block_details.get("previousBlockFinishPositions", {})
        next_positions = block_details.get("nextBlockHistoricalFinishPositions", {})

        # Combine data for each name in the block
        all_names = set(previous_tournaments.keys()) | set(previous_points.keys()) | \
                    set(next_field_points.keys()) | set(previous_positions.keys()) | set(next_positions.keys())
        
        for name in all_names:
            row = {
                "Full Name": name,
                "Previous Block Tournaments Played": previous_tournaments.get(name, "N/A"),
                "Previous Block Total Perk Cup Points": previous_points.get(name, "N/A"),
                "Next Block Historical Field Points": next_field_points.get(name, "N/A"),
                "Previous Block Finish Positions": previous_positions.get(name, "N/A"),
                "Next Block Historical Finish Positions": next_positions.get(name, "N/A"),
                "Tournament IDs": tournament_ids,
                "Block Draft Positions": block_draft_positions,
                "Block": block_number  # Include the block number
            }
            rows.append(row)

    # Save to CSV
    headers = ["Full Name", "Previous Block Tournaments Played", "Previous Block Total Perk Cup Points",
               "Next Block Historical Field Points", "Previous Block Finish Positions",
               "Next Block Historical Finish Positions", "Tournament IDs", "Block Draft Positions", "Block"]
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(rows)

    print(f"CSV saved to {output_file}")

# Load JSON data
file_path = "firestore_full_database.json"  # Replace with your actual file path
data = load_json(file_path)

# Extract blocks
blocks = data["blocks"]

# Save to a single CSV file
output_file = "CSV DATA/blocks_with_names.csv"
save_blocks_with_names(blocks, output_file)

import os
import pandas as pd

# Path to your CSV data directory
directory = 'CSV DATA'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Rename columns: replace spaces with underscores
        df.columns = [col.replace(' ', '_') for col in df.columns]
        
        # Save the modified DataFrame back to the same file
        df.to_csv(file_path, index=False)
        print(f"Processed: {filename}")
import pandas as pd

# Load the CSV files
events_scores_path = 'CSV DATA/all_event_scores.csv'
tournaments_path = 'CSV DATA/tournaments.csv'

# Read the data
events_scores_df = pd.read_csv(events_scores_path)
tournaments_df = pd.read_csv(tournaments_path)

# Normalize column names for joining
tournaments_df.rename(columns={
    'event_name': 'Event_Name',
    'tour': 'Tour',
    'year': 'Year',
    'event_id': 'Event_ID',
    'player_name': 'Player_Name'
}, inplace=True)

# Perform the merge (inner join based on Event_ID and Player_Name)
merged_df = pd.merge(
    events_scores_df,
    tournaments_df,
    on=['Event_ID', 'Player_Name'],
    how='inner',
    suffixes=('', '_duplicate')  # Avoid suffixes where possible
)

# Remove duplicated columns explicitly
columns_to_keep = [col for col in merged_df.columns if not col.endswith('_duplicate')]
merged_df = merged_df[columns_to_keep]
columns_to_keep
# Drop specified columns
merged_df.drop(columns=['finish_date', 'last_fetched', 'Final_Position'], inplace=True)

# Extract 'Month' and 'Day' from 'Date' column
merged_df['Month'] = pd.to_datetime(merged_df['Date']).dt.month
merged_df['Day'] = pd.to_datetime(merged_df['Date']).dt.day
merged_df.drop(columns=['Date'], inplace=True)

# Rename ambiguous columns to make them more descriptive (if needed)
merged_df.rename(columns={
    'Year': 'Event_Year',
    'Tour': 'Event_Tour',
    'Event_Name': 'Event_Name_Final'
}, inplace=True)

# Reorder columns to place 'Month' and 'Day' after 'Event_Year'
columns_order = ['Event_ID', 'Player_Name', 'Event_Year', 'Month', 'Day'] + \
    [col for col in merged_df.columns if col not in ['Event_ID', 'Player_Name', 'Event_Year', 'Month', 'Day']]
merged_df = merged_df[columns_order]

# Save the cleaned dataframe
merged_df.to_csv("CSV DATA/tours_events.csv", index=False)

import pandas as pd

# Load the datasets
players_info_path = 'CSV DATA/players_info.csv'
tours_events_path = 'CSV DATA/tours_events.csv'

players_info_df = pd.read_csv(players_info_path)
tours_events_df = pd.read_csv(tours_events_path)

# Harmonize column names for clarity
clear_column_names = {
    "sample_size": "SampleSize",
    "firstName": "FirstName",
    "lastName": "LastName",
    "countryCode": "CountryCode",
    "primary_tour": "PrimaryTour",
    "player_name": "PlayerName"
    # Add other renaming rules if needed
}

players_info_df.rename(columns=clear_column_names, inplace=True)

# Join the datasets on Player_Name/PlayerName
combined_df = pd.merge(
    tours_events_df,
    players_info_df,
    left_on="Player_Name",
    right_on="PlayerName",
    how="left"
)

# Save the combined dataframe to a CSV file
combined_df.to_csv("CSV DATA/tours_events_players.csv", index=False)
file_path = "CSV DATA/tours_events_players.csv"
df = pd.read_csv(file_path)

# Clean up column names
df.columns = [
    col.strip().replace(" ", "_").lower()  # Normalize column names
    for col in df.columns
]

# Save the cleaned DataFrame back to a CSV file if needed
# Drop columns starting with specific prefixes
prefixes_to_drop = ('100', '150', '50', 'under', 'over' , "playerskillratings")
columns_to_drop = [col for col in df.columns if col.startswith(prefixes_to_drop)]
df = df.drop(columns=columns_to_drop)

# Save the cleaned DataFrame
cleaned_file_path = "CSV DATA/tours_events_players.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned CSV saved to {cleaned_file_path}")
print("combined tours_events_players is saved ")
df

CSV file 'CSV DATA/tournaments.csv' has been created.
CSV file 'CSV DATA/players_info.csv' has been created.
All event scores data saved to CSV DATA/all_event_scores.csv
CSV saved to CSV DATA/blocks_with_names.csv
Processed: all_event_scores.csv


/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:279: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed: tours_events.csv


/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:279: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed: tours_events_players.csv


...

/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:279: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed: tours_events_players_cleaned.csv
Processed: players_info.csv


/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:295: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tournaments_df = pd.read_csv(tournaments_path)
/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:349: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  tours_events_df = pd.read_csv(tours_events_path)
/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_5780/1108676945.py:376: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Cleaned CSV saved to CSV DATA/tours_events_players.csv
combined tours_events_players is saved 


,event_id,player_name,event_year,month,day,event_name_final,event_tour,tournament_id,total_score,strokes_gained,...,total_fit_adjustment,final_pred,country_x,sg_app_pred,driving_accuracy_adjustment,playername,sg_ott_y,amateur_status,primarytour,comments
0,1,"Schwartzel, Charl",2022,6,11,London,liv,1,203,13.188,...,NaN,NaN,South Africa,NaN,NaN,"Schwartzel, Charl",NaN,0,LIV Tour,NaN
1,1,"Uihlein, Peter",2022,6,11,London,liv,1,205,11.188,...,NaN,NaN,United States,NaN,NaN,"Uihlein, Peter",NaN,0,LIV Tour,NaN
2,1,"Grace, Branden",2022,6,11,London,liv,1,205,11.188,...,NaN,NaN,South Africa,NaN,NaN,"Grace, Branden",NaN,0,LIV Tour,NaN
3,1,"Horsfield, Sam",2022,6,11,London,liv,1,207,9.188,...,NaN,NaN,England,NaN,NaN,"Horsfield, Sam",NaN,0,LIV Tour,NaN
4,1,"Johnson, Dustin",2022,6,11,London,liv,1,209,7.188,...,NaN,NaN,United States,0.026,NaN,"Johnson, Dustin",0.244,0,LIV Tour,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39279,493,"Coody, Pierceson",2024,11,24,The RSM Classic,pga,493,151,-6.761,...,-0.132317,-0.218662,United States,-0.390,-0.009684,"Coody, Pierceson",0.130,Cond 126-150,PGA TOUR,NaN
39280,493,"Kim, Si Woo",2024,11,24,The RSM Classic,pga,493,70,1.026,...,0.060509,0.916700,Korea - Republic of,0.576,0.023855,"Kim, Si Woo",0.319,Top 50,PGA TOUR,NaN
39281,493,"Power, Seamus",2024,11,24,The RSM Classic,pga,493,76,-4.974,...,0.006553,0.805881,Ireland,0.152,0.010088,"Power, Seamus",0.110,Top 51-60,PGA TOUR,NaN
39282,493,"Power, Seamus",2024,11,24,The RSM Classic,pga,493,76,-4.974,...,0.006553,0.805881,Ireland,0.152,0.010088,"Power, Seamus",0.110,Top 51-60,PGA TOUR,NaN


# To SQL Database if needed

In [22]:
import sqlite3
import pandas as pd

# Load CSVs into DataFrames
tournaments = pd.read_csv('events_and_players.csv')
#events = pd.read_csv('blocks_with_names.csv')  # Assuming this is the "events" data
#players = pd.read_csv('players_info.csv')
#blocks = pd.read_csv('blocks_with_names.csv')

# Connect to SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect('golf_database.db')
cursor = connection.cursor()

# Write DataFrames to SQLite tables
tournaments.to_sql('events_and_players', connection, if_exists='replace', index=False)
#events.to_sql('blocks', connection, if_exists='replace', index=False)
#players.to_sql('players', connection, if_exists='replace', index=False)
#blocks.to_sql('blocks', connection, if_exists='replace', index=False)

# Close the connection
connection.close()

print("Data successfully saved to golf_database.db")

/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_14006/403102381.py:5: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  tournaments = pd.read_csv('events_and_players.csv')


Data successfully saved to golf_database.db
